# Homework 1 (Due 5:30pm PST March 19th, 2019): Naive Bayes and Probability

**A. Compute the following probabilities (4 points):**
```python
corpus = [("I love going to class", "ham"),
("Class is going well", "ham"),
("Do you love bargains", "spam"),
("Act now for bargains", "spam"),
("I love you", "ham")]
```
* $P(x = love | y = ham)$
* $P(y = spam)$
* $P(x = bargains)$
* $P(y = ham | x = "You love bargains")$

**B. Former In-Class Exercise Word Count (Now Homework 1B) - 6 points**

You are a business analyst working for a major US toy retailer:

* Part 1: A manager in the marketing department wants to find out the most frequently used words in positive reviews (five stars) and negative reviews (one star) in order to determine who the toys are being bought for (sons, significant others, grandchildren, siblings, self, etc.). He would like your opinion on which customer segments' (son, daughter, granddaugher, grandson, niece, nephew, sibling, significant other, etc.) marketing messaging tends to outperform other segments.

* Part 2: One of your product managers suspects that **toys with batteries tend to be significantly lower rated than toys that do not require batteries**. She would like to see some data points confirming or rejecting her hypothesis. 

Perform the same word count analysis using the reviews received from Amazon to answer your marketing manager's and product manager's question. They are stored in two files, (`poor_amazon_toy_reviews.txt`) and (`good-amazon-toy-reviews.txt`). **Provide only few sentences with your findings and business recommendations for each part (Part A and B.** Make any assumptions you'd like to- this is a fictitious company after all. I just want you to get into the habit of "finishing" your analysis: to avoid delivering technical numbers to a non-technical manager.

**C. Read "What is Natural Language Processing? The Business Use Case Explained" (CIO.com) and Intro to Algorithmic Marketing (pages 179 - 184, 193 - 201)**. Be prepared for a short in-class graded exercise at the beginning of Week 2 on the topics in these readings (read it once, don't read it twice).

### **Submit everything as a new notebook, either via Slack direct message to me, or email to me (ychen220@usc.edu). Much of the code should already be written for you!**

### A. Compute the following probabilities (4 points):

P(x = love | y = ham) = 2/3

P(y = spam) = 2/5

P(x = bargains) = 2/5

P(y = ham | x = "You love bargains") = 0

### B. Former In-Class Exercise Word Count (Now Homework 1B) - 6 points

* Part 1: A manager in the marketing department wants to find out the most frequently used words in positive reviews (five stars) and negative reviews (one star) in order to determine who the toys are being bought for (sons, significant others, grandchildren, siblings, self, etc.). He would like your opinion on which customer segments' (son, daughter, granddaugher, grandson, niece, nephew, sibling, significant other, etc.) marketing messaging tends to outperform other segments.

In [ ]:
import os
os.getcwd()

In [ ]:
good= open("good_amazon_toy_reviews.txt", "r")
poor = open("poor_amazon_toy_reviews.txt", "r")

In [ ]:
good.seek(0)
poor.seek(0)
good=good.readlines()
poor=poor.readlines()

In [ ]:
segments=['son', 'daughter', 'granddaugher', 'grandson', 'niece', 'nephew', 'sibling','siblings']
seg_dict_good={key:0 for key in segments}

In [ ]:
import nltk
for line in good:
    words=line.split()
    for word in words:
        if word in segments:
            seg_dict_good[word]+=1

In [ ]:
import pandas as pd
seg_good_df=pd.DataFrame(list(seg_dict_good.items()), columns=['Segment','Frequency'])
seg_good_df.sort_values(by='Frequency',ascending=False)

In [ ]:
seg_dict_poor={key:0 for key in segments}

In [ ]:
for line in poor:
    words=line.split()
    for word in words:
        if word in segments:
            seg_dict_poor[word]+=1

In [ ]:
seg_poor_df=pd.DataFrame(list(seg_dict_poor.items()), columns=['Segment','Frequency'])
seg_poor_df.sort_values(by='Frequency',ascending=False)

In [ ]:
seg_good_df['Frequency']/seg_poor_df['Frequency']

Most good reviews come from those who bought the toy for their sons (5640 times) and daughters (5541 times). Quite a number of customers who rated it high bought it for their grandsons (2491 times) rather than their granddaughters. 

Most poor reviews come from those who bought it for their sons (504 times) rather than for their daughters (327 times). Some who rated it low bought it for their grandsons (113 times)

To summarize, if only looking at the ratios of good reviews over poor reviews, people who bought the toy for their sons tend to be more picky. The marketing messaging seems to convey better to customers who bought it for their daughters or their grandsons. 

#### Part 2: One of your product managers suspects that toys with batteries tend to be significantly lower rated than toys that do not require batteries. She would like to see some data points confirming or rejecting her hypothesis. 

* Null Hypothesis: Toys with and without batteries are equally rated, i.e. toys with and without batteries have the same probability of receiving negative reviews.
* Alternative Hypothesis: Toys with batteries will received more negative reviews than toys without batteries.

In [ ]:
import re
good_battery=[w for w in good if re.search('[A-Za-z0-9_]*battery[A-Za-z0-9_]*', w) \
                or re.search('[A-Za-z0-9_]*batteries[A-Za-z0-9_]*', w)]

In [ ]:
poor_battery=[w for w in poor if re.search('[A-Za-z0-9_]*battery[A-Za-z0-9_]*', w) \
                or re.search('[A-Za-z0-9_]*batteries[A-Za-z0-9_]*', w)]

In [ ]:
#probability of seeing "battery" or "batteries" in a review
prob_battery=(len(good_battery)+len(poor_battery))/(len(good)+len(poor))

In [ ]:
prob_battery

In [ ]:
#probability of "battery"/"batteries" and negative
prob_battery_poor=len(poor_battery)/(len(good)+len(poor))

In [ ]:
prob_battery_poor

In [ ]:
#probability of no "battery"/"batteries" and negative
prob_no_battery_poor=(len(poor)-len(poor_battery))/(len(good)+len(poor))

In [ ]:
prob_no_battery_poor

In [ ]:
#probability of not seeing "battery" or "batteries" in a review
prob_no_battery=1-prob_battery

In [ ]:
prob_no_battery

In [ ]:
#probability of a negative review given "battery"/"batteries" appears
prob1=prob_battery_poor/prob_battery
prob2=prob_no_battery_poor/prob_no_battery

In [ ]:
num_battery=len(good_battery)+len(poor_battery)
num_no_battery=len(good)+len(poor)-(len(good_battery)+len(poor_battery))

In [ ]:
p_pool=(num_battery*prob1+num_no_battery*prob2)/(num_battery+num_no_battery)

In [ ]:
ste=((p_pool*(1-p_pool)/num_battery)+(p_pool*(1-p_pool)/num_no_battery))**0.5

In [ ]:
z_score=((prob1-prob2)-0)/ste

In [ ]:
from scipy import stats
p_value = stats.norm.sf(abs(z_score)) #one-sided

In [ ]:
p_value<0.05

Since p_value is smaller than 0.05, meaning that the probability of getting new results as or more extreme than the actual data is less than 5%, if the null hypothesis were true. We reject the null hypothesis at 5% significance level and conclude that toys with batteries will received more negative reviews than toys without batteries. 

Therefore, when producing toys with batteries, we should be more careful with quality control. People are more likely give a negative review when there is a battery issue.